### Getting webcam images

In [ ]:
import cv2

cap = cv2.VideoCapture(0) # video capture source camera (here webcam of laptop)
num = 0

while cap.isOpened():

    succes, img = cap.read()
    k = cv2.waitKey(3)

    if k == 30:
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('frames/img' + str(num) + '.png', img)
        print("image has been saved")
        num += 1
    cv2.imshow('webcam', img)

cap.release()
cv2.destroyAllWindows()

### Finding chessboard corners

In [15]:
import numpy as np
import cv2 as cv
import glob

chessboardSize = (10, 7) # Number of inner corners per a chessboard width and height
frameSize = (640, 480) # Resolution of images

# Termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


images = glob.glob('frames/*.png')

for image in images:

    img = cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # Convert to grayscale for findChessboardCorners()

    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None) # Find the corners

    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria) # Refine the corners
        imgpoints.append(corners)

        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('finding corners', img)
        cv.waitKey(1000)

cv.destroyAllWindows()


### Calibrate camera

In [18]:
import pickle

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Camera matrix: \n")
print(cameraMatrix)
print("Distortion coefficients: \n")
print(dist)
print("Rotation vectors: \n")
print(rvecs)
print("Translation vectors: \n")
print(tvecs)

pickle.dump((cameraMatrix, dist), open("output\camera_calibration_1\calibration_webcam1.pkl", "wb"))
pickle.dump(cameraMatrix, open("output\camera_calibration_1\cameraMatrix_webcam2.pkl", "wb"))
pickle.dump(dist, open("output\camera_calibration_1\dist_webcam1.pkl", "wb"))

Camera matrix: 

[[624.80025092   0.         340.91494291]
 [  0.         624.00862802 280.26098028]
 [  0.           0.           1.        ]]
Distortion coefficients: 

[[ 0.03660126 -2.26094412  0.02942902  0.01692354 10.84549437]]
Rotation vectors: 

(array([[ 0.24623066],
       [ 0.14468993],
       [-3.00279582]]), array([[-0.09368653],
       [ 0.0236585 ],
       [ 2.55354513]]), array([[-0.03930321],
       [-0.09996256],
       [ 2.31769303]]), array([[-0.04163459],
       [-0.15769417],
       [ 2.18661318]]), array([[ 0.47792882],
       [-0.42787597],
       [-2.75438703]]), array([[ 0.06872082],
       [-0.21240927],
       [ 1.89748745]]), array([[ 0.03101564],
       [-0.06467237],
       [ 2.27459391]]), array([[-0.06940139],
       [-0.0275615 ],
       [ 2.64799623]]), array([[-0.47331147],
       [-0.32937628],
       [-2.61374298]]), array([[-0.29706216],
       [ 0.03507936],
       [-2.83469866]]))
Translation vectors: 

(array([[ 7.01383701],
       [ 2.7412705